In [1]:
import numpy as np
import os as os
import tifffile as tif
import cv2

from remove_isolated_px import remove_isolated_px 
from smoothing import smoothing

In [2]:
#Set path until info about dataset number and path to input
path_num = '/Volumes/Vin/Python/shortTrAn/Trc'
path_input = '/' #allows the selection of a subfolder in which binned data are contained;
#last sign has to be a slash /

#Number of to be evaluated datasets
N = 20

#Parameter for Filter1: Removal of pixels containing less entries than the threshold.
threshold = 6 

#Parameter for Filter2: Removal of isolated pixels.
kdim = 3 #odd number, defines folding kernel . Largest number for cross kernel is 5.
kshape='box' #cross or box (kernel). 

#Parameter for Filter3: Smoothing out.
kdim_smoothing= 3 #odd number defines smoothing kernel.
wdth_outliers = 2 #defines width of outlier.
mode_outliers = 'gauss' #or 'remove'

In [3]:
for Nu in range(1,N+1):
    path = path_num + str(Nu) + path_input

    scaled_pointfield=tif.imread(path +'scaled_pointfield.tif')
    scld_ten_xx=tif.imread(path+'scld_ten_xx.tif')
    scld_ten_xy=tif.imread(path+'scld_ten_xy.tif')
    scld_ten_yy=tif.imread(path+'scld_ten_yy.tif')
    scld_ten_yx=tif.imread(path+'scld_ten_yx.tif')
    scld_vec_x=tif.imread(path+'scld_vec_x.tif')
    scld_vec_y=tif.imread(path+'scld_vec_y.tif')

    if not os.path.exists(path+'results_filtering'):
        #print('Path does not exist')
        os.mkdir(path+'results_filtering')
        directory = path+'results_filtering/'
    else:
        directory = path+'results_filtering/'

    #Filter #1 and #2. Removes all pixels which have less entries then threshold x and are isolated.
    scld_count_mskd,scld_ten_xx_mskd,scld_ten_xy_mskd,scld_ten_yx_mskd,scld_ten_yy_mskd,scld_vec_x_mskd, \
        scld_vec_y_mskd=remove_isolated_px(kdim,kshape,threshold,directory,scaled_pointfield,scld_ten_xx,
                                           scld_ten_xy,scld_ten_yx,scld_ten_yy,scld_vec_x,scld_vec_y)

    #Filter #3. Smoothes entries to the surrounding pixels using a for contributing entries weighted average filter 
    #of an certain kernel size kdim.
    scld_count_fil,scld_vec_x_fil,scld_vec_y_fil,scld_ten_xx_fil,scld_ten_xy_fil,scld_ten_yx_fil,scld_ten_yy_fil\
    =smoothing(kdim_smoothing,wdth_outliers,mode_outliers,directory,scaled_pointfield,scld_count_mskd,scld_vec_x_mskd, 
               scld_vec_y_mskd,scld_ten_xx_mskd,scld_ten_xy_mskd,scld_ten_yx_mskd,scld_ten_yy_mskd)

    #Saves input parameter in text file:
    txt = open(directory +'input_parameter.txt',"w+")
    txt.write(path_num+'N\n')
    txt.write(path_input +'\n') 
    txt.write('Number of datasets: '+str(N) + '\n')
    txt.write('Filter 1: Threshold '+str(threshold) + '\n') 
    txt.write('Filter 2: Kernel dimension '+str(kdim) + ' and shape ' + kshape + ' kernel \n')  
    txt.write('Filter 3: Kernel size '+str(kdim_smoothing)+', width outlier '+str(wdth_outliers)+', '+mode_outliers)
    txt.close()
    
    print('Finished cell ' +str(Nu))


Finished cell 1
Finished cell 2
